# Masked Language Modeling for Word Sense Induction

In this notebook, I have implemented a WSI algorithm inspired by the papers https://arxiv.org/pdf/1905.12598.pdf and https://www.aclweb.org/anthology/S19-2004.pdf. It is, due to time restrictions, a somewhat simplified version of those approaches. The algorithm is as follows:
- for a set of sentences $S$ where each sentence $s_{i}$ contains some target word $w$ at least once, create a new set $S'$ by inserting a pattern that contains the [MASK] token after every occurence of $w$ (I chose the pattern __( или даже [MASK] )__ based on my preliminary research; the same pattern was found to be most effective in https://arxiv.org/pdf/1905.12598.pdf)
- for each [MASK] token in $s'_{i}$, compute a logit vector over the BERT wordpiece vocabulary (I found it works better than post-softmax probability during my preliminary research)
- compute the same vector in the position of each occurrence of $w$ in the original sentence $s_{i}$
- extract top $2n$ most probable candidates for every position and concatenate them, including the logits
- lemmatize the candidates and leave top $n$ most probable candidates for every sentence
- represent each sentence in the set with sparse $tf-idf$ vectors constructed from these candidates
- cluster these sparse representations with Agglomerative Clustering using cosine distance and complete linkage (I chose to infer the optimal number of clusters, one that yields the highest silhouette score, from the data)

NB! I chose to use the vanilla RuBERT for this approach because it seems to work better.

In [1]:
# !pip install transformers

In [2]:
import warnings
from tqdm import tqdm
from typing import List, Tuple, Set, Dict, Iterator, Iterable

import numpy as np
import pandas as pd
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import RegexpTokenizer

import torch
from transformers import BertTokenizer, BertForMaskedLM

from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score, silhouette_score

warnings.filterwarnings('ignore')

In [3]:
re_tokenizer = RegexpTokenizer(r'[А-Яа-яЁё]+|[A-za-z]+|\w+|[«»\'",.:;!?\(\)-–—]|[^\w\s]+')
word_tokenize = re_tokenizer.tokenize

In [4]:
RUBERT_PATH = '../../RuBERT/rubert_cased_L-12_H-768_A-12_pt'
USE_GPU = False

tokenizer = BertTokenizer.from_pretrained(RUBERT_PATH, do_lower_case=False)
model = BertForMaskedLM.from_pretrained(RUBERT_PATH)
if USE_GPU:
    model = model.cuda()
model.eval()
device = torch.device('cuda:0' if USE_GPU else 'cpu')

In [5]:
CLS_ID = tokenizer.vocab[tokenizer.cls_token]
SEP_ID = tokenizer.vocab[tokenizer.sep_token]
MASK = tokenizer.mask_token

PATTERN = ['(', 'или', 'даже', MASK, ')']

In [6]:
def apply_pattern(sentence: str, char_positions: List[Tuple[int, int]]) -> Tuple[List[str], List[str], Set[str]]:
    relevant_tokens = {sentence[pos[0]:pos[1]-1] for pos in char_positions}
    
    tokens = word_tokenize(sentence)
    modified_tokens = []
    
    for i, token in enumerate(tokens):
        modified_tokens.append(token)
        if token in relevant_tokens:
            modified_tokens.extend(PATTERN)
    
    return tokens, modified_tokens, relevant_tokens

In [7]:
' '.join(apply_pattern('Доброе утро! Как вы поживаете? Сегодня чудесная погода и прекрасный день, не находите?', [(68, 73)])[1])

'Доброе утро ! Как вы поживаете ? Сегодня чудесная погода и прекрасный день ( или даже [MASK] ) , не находите ?'

In [8]:
class Lemmatizer:
    def __init__(self):
        self.morph = MorphAnalyzer()
        self.cache = {}
    
    def lemmatize(self, token):
        if token in self.cache:
            return self.cache[token]
        norm = self.morph.parse(token)[0].normal_form
        self.cache[token] = norm
        return norm

lemmatizer = Lemmatizer()
lemmatize = lemmatizer.lemmatize

In [9]:
def get_token_logits(tokens: List[str], relevant_tokens: Set[str]) -> np.ndarray:
    input_indices, wordpiece_positions, acc_len = [], [], 1
    for token in tokens:
        indices = tokenizer.encode(token, add_special_tokens=False)
        input_indices.extend(indices)
        
        if token in relevant_tokens:
            wordpiece_positions.append(acc_len)
        
        acc_len += len(indices)
    
    with torch.no_grad():
        model_output = model(torch.tensor([[CLS_ID] + input_indices + [SEP_ID]]).to(device))[0][0]
    
    return model_output[wordpiece_positions].detach().cpu().numpy()

In [10]:
def extract_candidates(logits: np.ndarray, n: int) -> List[str]:
    top_candidates = [(tokenizer.convert_ids_to_tokens(int(id)), row[int(id)])
                      for row in logits for id in row.argsort()[-2*n:]]
    
    filtered_candidates = set()
    for word, _ in sorted(top_candidates, key=lambda x: x[1], reverse=True):
        filtered_candidates.add(lemmatize(word))
        if len(filtered_candidates) == n:
            break
    
    return list(filtered_candidates)

In [11]:
def get_candidates(sentence: str, char_positions: List[Tuple[int, int]], n: int) -> List[str]:
    _, modified_tokens, _ = apply_pattern(sentence, char_positions)
    logits = get_token_logits(modified_tokens, {MASK,})
    
    return extract_candidates(logits, n)

In [12]:
get_candidates('- Доброе утро! Как вы поживаете? Сегодня чудесная погода и прекрасный день, не находите? - Да, сегодня отличный день.',
               [(70, 75), (112, 117)], n=10)

['прекрасный',
 'утро',
 'отличный',
 'ночь',
 'вечер',
 'праздник',
 'день',
 'воскресение',
 'хороший',
 'выходной']

In [13]:
dummy = lambda x: x

def create_tfidf_matrix(data: List[Tuple[str, List[Tuple[int, int]], str, int]], n: int = 50) -> np.ndarray:
    candidate_set = [get_candidates(sent, pos, n) for sent, pos, _, _ in data]
    
    tfidf = TfidfVectorizer(preprocessor=dummy, tokenizer=dummy, lowercase=False)
    return tfidf.fit_transform(candidate_set).todense()

In [14]:
data = pd.read_csv('https://raw.githubusercontent.com/nsykhr/russe-wsi-kit/master/data/main/active-dict/train.csv', sep='\t')
data.dropna(subset=['positions'], inplace=True) # let's drop the rows where the relevant token is not present in the context
data.positions = data.positions.apply(lambda x: [tuple(map(int, pos.split('-'))) for pos in x.split(',')])
data

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,дар,1,NaN,"[(18, 22)]",Отвергнуть щедрый дар
1,2,дар,1,NaN,"[(21, 28)]",покупать преданность дарами и наградами
2,3,дар,1,NaN,"[(19, 23)]",Вот яд – последний дар моей Изоры
3,4,дар,1,NaN,"[(81, 87)]",Основная функция корильных песен – повеселить ...
4,5,дар,1,NaN,"[(151, 157)]",Но недели две спустя (Алевтина его когда-то об...
...,...,...,...,...,...,...
2068,2069,зонт,1,NaN,"[(85, 91)]","Такая погода легко переживается весной, а вот ..."
2069,2070,зонт,2,NaN,"[(8, 13)]",Пляжный зонт
2070,2071,зонт,2,NaN,"[(18, 25)]",сидеть в кафе под зонтом
2071,2072,зонт,2,NaN,"[(21, 29)]","Cтолики под широкими зонтами, несколько привин..."


In [15]:
MAX_CLUSTERS = 8

def find_optimal_clustering(estimator_class, X, kwargs) -> int:
    best_score, best_clustering = -1, None

    for n_clusters in range(2, min(MAX_CLUSTERS, len(X) - 1)):
        kwargs['n_clusters'] = n_clusters
        estimator = estimator_class(**kwargs)
        preds = estimator.fit_predict(X)
        score = silhouette_score(X, preds)

        if score > best_score:
            best_score = score
            best_clustering = preds

    return best_clustering

In [16]:
def cluster_and_evaluate(data: pd.DataFrame):
    data_by_words = {key: [] for key in data.word.unique()}
    for i, row in data.iterrows():
        data_by_words[row.word].append((row.context, row.positions, row.gold_sense_id, i))
    
    aggl = AgglomerativeClustering
    kwargs = {'affinity': 'cosine', 'linkage': 'complete'}

    print('word\tARI\tcount')

    score = 0
    size = sum(len(word_data) for word_data in data_by_words.values())

    for word, word_data in data_by_words.items():
        labels = [id for _, _, id, _ in word_data]
        tfidf_matrix = create_tfidf_matrix(word_data, n=50)
        preds = find_optimal_clustering(aggl, tfidf_matrix, kwargs)

        ari = adjusted_rand_score(labels, preds)
        print(f'{word}\t{ari}\t{len(word_data)}')

        score += ari * len(word_data) / size

    print(f'\noverall\t{score}\t{size}')

In [17]:
cluster_and_evaluate(data)

word	ARI	count
дар	0.07864534830340632	36
двигатель	0.47093023255813954	14
двойник	0.02067464635473339	25
дворец	0.691699604743083	13
девятка	0.12519370501739038	47
дедушка	0.55	9
дежурная	-0.11594202898550726	12
дежурный	0.07142857142857137	13
декабрист	0.07903549899531145	11
декрет	0.664819944598338	12
дело	0.06755265427538187	129
демобилизация	0.7144581221337195	14
демократ	0.20872250096487846	18
демонстрация	0.3424030650253645	37
дерево	0.38636363636363635	21
держава	0.050131926121372065	15
дерзость	0.08514621787175156	37
десятка	-0.008636525998877391	36
десяток	-0.007067137809187302	20
деятель	0.7130730050933786	14
диалог	0.22980251346499103	14
диаметр	0.016528925619834742	18
диплом	0.14788097385031554	25
директор	-0.07843137254901954	11
диск	0.19577812912780693	62
дичь	0.04198895027624308	18
длина	0.021709633649932215	21
доброволец	0.18331805682859767	12
добыча	0.37547647195306544	35
доказательство	0.03937350445943006	24
доктор	1.0	17
долгота	0.09684210526315788	13
доля	0.3307596

# Prediction

## Main

In [18]:
data = pd.read_csv('https://raw.githubusercontent.com/nsykhr/russe-wsi-kit/master/data/main/active-dict/test-solution.csv', sep='\t')
data.dropna(subset=['positions'], inplace=True) # let's drop the rows where the relevant token is not present in the context
data.positions = data.positions.apply(lambda x: [tuple(map(int, pos.split('-'))) for pos in x.split(',')])
data

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,2074,давление,1,NaN,"[(0, 9)]",Давление пара создается движением поршня в цил...
1,2075,давление,2.2,NaN,"[(13, 22)]","«У тебя что, давление поднялось?» Я сказал, чт..."
2,2076,давление,2.2,NaN,"[(56, 65)]",Я жалуюсь Никоновичу наконец на головокружение...
3,2077,давление,2.1,NaN,"[(0, 9)]",Давление в котле не менялось
4,2078,давление,2.2,NaN,"[(25, 34)]",Он каждые два часа мерил давление и сахар в крови
...,...,...,...,...,...,...
3724,5798,зуд,2,NaN,"[(43, 47)]",Многих американцев одолевает романтический зуд...
3725,5799,зуд,2,NaN,"[(23, 27)]",Если на нее не находил зуд рассказывания истор...
3726,5800,зуд,2,NaN,"[(27, 33)]","С раздражающей завистью, с зудом неудовлетворе..."
3727,5801,зуд,2,NaN,"[(12, 16)]",Нестерпимый зуд любопытства


In [19]:
def cluster_and_write_to_pandas(data: pd.DataFrame):
    data_by_words = {key: [] for key in data.word.unique()}
    for i, row in data.iterrows():
        data_by_words[row.word].append((row.context, row.positions, row.gold_sense_id, i))
    
    aggl = AgglomerativeClustering
    kwargs = {'affinity': 'cosine', 'linkage': 'complete'}

    for word, word_data in tqdm(data_by_words.items()):
        tfidf_matrix = create_tfidf_matrix(word_data, n=50)
        preds = find_optimal_clustering(aggl, tfidf_matrix, kwargs)

        for pred, row in zip(preds, word_data):
            data.predict_sense_id.loc[row[-1]] = pred

In [20]:
cluster_and_write_to_pandas(data)
data.to_csv('../data/main/active-dict/result_mlm_aggl.csv', sep='\t', index=None)

100%|██████████| 168/168 [21:03<00:00,  7.52s/it]


The algorithm achieves __0.190087__ ARI on the test data. Not quite as good as the other approach I tried, but still significantly better than the baseline. Let's see how it does on the RuTenTen data.

## Additional

In [21]:
data = pd.read_csv('https://raw.githubusercontent.com/nsykhr/russe-wsi-kit/master/data/additional/active-rutenten/train.csv', sep='\t')
data.dropna(subset=['positions'], inplace=True) # let's drop the rows where the relevant token is not present in the context
data.positions = data.positions.apply(lambda x: [tuple(map(int, pos.split('-'))) for pos in x.split(',')])
data.positions = data.positions.apply(lambda x: [(a, b+2) for a, b in x])
data

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,альбом,2,NaN,"[(88, 96)]",достаточно лишь колесиком мышки крутить вниз. ...
1,2,альбом,3,NaN,"[(85, 93)]","выступал в составе команды с таким названием, ..."
2,3,альбом,2,NaN,"[(81, 89)]",". Работает так себе, поскольку функция заточен..."
3,4,альбом,3,NaN,"[(84, 91)]",одержала победу в двух из пяти номинаций: 'Луч...
4,5,альбом,3,NaN,"[(83, 90)]",встречи с Божественным. Вы испытаете ни с чем ...
...,...,...,...,...,...,...
3666,3667,группа,4,NaN,"[(102, 109)]","напротив, цветет пуще прежнего. География расш..."
3667,3668,группа,4,NaN,"[(93, 100)]","синтетической работе, терпение и упорство, жел..."
3668,3669,группа,4,NaN,"[(20, 27)]",Маркетинг процедуры.Группа компаний Кивеннапа ...
3669,3670,группа,4,NaN,"[(100, 107)]",International» признались миллионам слушателей...


In [22]:
data.context.loc[494] = '15, 000 на всех членов семьи.Вот тогда и законы будут человечными Анатомия и физиология человека'
data.positions.loc[494] = [(66, 75)]

data.context.loc[1202] = 'выше чем 1 метр или до 12 лет-8 евро, дети ниже чем 1 метр –вход бесплатный Билеты: взрослые-16 евро, дети выше чем 1 метр или старше 10 лет-12 евро, дети до 10 лет'
data.positions.loc[1202] = [(76, 83)]

data.context.loc[1381] = 'крепление Крепление на стену по стандарту VESA 100мм Блок питания внешний'
data.positions.loc[1381] = [(53, 58)]

data.context.loc[1398] = 'Пористые заполнители Блоки оконные'
data.positions.loc[1398] = [(21, 27)]

data.context.loc[1432] = 'СКАТ БЛОКИ ПИТАНИЯ БПУ-24-0,5; БПУ-24-0,7; БПУ-12 -1,5; БПС -12 -0,7; БП-TV1; БП-TV3 БЛОК ПИТАНИЯ СЕТЕВОЙ БПС СИСТЕМА ДИСТАНЦИОННОГО... БПС -12 . Блок питания с симисторами'
data.positions.loc[1432] = [(85, 90)]

data.context.loc[1514] = 'Библиографические ресурсы и каталоги Блок библиографических ресурсов глобальных сетей обширен и разнообразен. Его главной'
data.positions.loc[1514] = [(37, 42)]

data.context.loc[2019] = 'Выпускаемая продукция Вешалка детская'
data.positions.loc[2019] = [(22, 30)]

data.context.loc[2566] = 'Электропроводка для подключения светодиодных знаков в задней части прицепа вилка/розетка) для подключения электрооборудования прицепа к электросети автомобиля'
data.positions.loc[2566] = [(75, 81)]

data.context.loc[2811] = 'Волги только левым расположением запасного колеса. Оно так же прикручено винтом. горизонтальным торсионам и удерживалась ими в открытом положении. Причем оригинальной'
data.positions.loc[2811] = [(73, 80)]

In [23]:
cluster_and_write_to_pandas(data)
data.to_csv('../data/additional/active-rutenten/result_mlm_aggl.csv', sep='\t', index=None)

100%|██████████| 20/20 [36:31<00:00, 109.60s/it] 


The achieved ARI is __0.188276__. I wasn't able to beat the baseline on this dataset yet, but it is definitely possible if one invests some more time into refining every component of this approach.